# Sequence Models & How to use them

**학습목표**
1. Bidirectional layer를 이해한다.
2. Bidirectional layer를 사용할 줄 안다.

-----------------
* Thanks to : 한기영 대표님 @ Data Insight

## 1.환경 및 데이터 준비

### Import Packages

In [78]:
#라이브러리들을 불러오자.
#라이브러리들을 불러오자.
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd




### Data Loading

In [79]:
url = 'https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/Stock_Edwards_Lifesciences_corporation.csv'

# 판다스로 데이터를 불러오시오.
data = pd.read_csv(url)

In [80]:
data.head()

,Date,Open,High,Low,Close,Adj_Close,Volume
0,2000/3/27,3.812500,4.156250,3.812500,4.125000,4.125000,3675600
1,2000/3/28,4.125000,4.125000,4.000000,4.015625,4.015625,1077600
2,2000/3/29,4.000000,4.031250,3.953125,4.000000,4.000000,437200
3,2000/3/30,4.000000,4.000000,3.843750,3.843750,3.843750,1883600
4,2000/3/31,3.734375,3.734375,3.390625,3.390625,3.390625,7931600


##3.데이터 준비

### Date 컬럼을 제거하시오.

In [81]:
data = data.drop('Date', axis = 1)

In [82]:
data.head()

,Open,High,Low,Close,Adj_Close,Volume
0,3.812500,4.156250,3.812500,4.125000,4.125000,3675600
1,4.125000,4.125000,4.000000,4.015625,4.015625,1077600
2,4.000000,4.031250,3.953125,4.000000,4.000000,437200
3,4.000000,4.000000,3.843750,3.843750,3.843750,1883600
4,3.734375,3.734375,3.390625,3.390625,3.390625,7931600


## 4.Sequence 데이터 구조로 만들기

**조건**
* 내일의 Close를 예측할 것이다. 이를 Y로 둘 것.
* timestep은 10주를 본다. (주식은 5일이 1주일)
* 맞추어 전처리

**데이터 분할 규칙**
* 가장 최근 1주일을 테스트 데이터로 둔다.
* 테스트 데이터를 제외하고, 가장 최근 2주를 벨리데이션 데이터로 둔다.

In [83]:
data.corr()

,Open,High,Low,Close,Adj_Close,Volume
Open,1.000000,0.999907,0.999899,0.999806,0.999806,0.048770
High,0.999907,1.000000,0.999874,0.999909,0.999909,0.051444
Low,0.999899,0.999874,1.000000,0.999912,0.999912,0.045101
Close,0.999806,0.999909,0.999912,1.000000,1.000000,0.047917
Adj_Close,0.999806,0.999909,0.999912,1.000000,1.000000,0.047917
Volume,0.048770,0.051444,0.045101,0.047917,0.047917,1.000000


In [103]:
data.head()

AttributeError: ignored

In [85]:
data = data.drop('Volume', axis = 1)

In [86]:
# 판다스 데이터 프레임을 넘파이로 옮김.
data = data.values

In [87]:
timestep= 50

x = [ ]  # x를 담을 빈 공간
y = [ ]  # y를 담을 빈 공간

x = np.array([data[i:i+timestep] for i in range(len(data) - timestep)])
y = np.array([data[i+timestep, -2] for i in range(len(data) - timestep)])

x_train, x_val, x_test = x[:-15], x[-15:-5], x[-5:]
y_train, y_val, y_test = y[:-15], y[-15:-5], y[-5:]

print(x.shape, y.shape)
print('-------------------------------')
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)
print('-------------------------------')
print('[ # , timestep, feature수 ] <-- 데이터의 구조 : ')

(4342, 50, 5) (4342,)
-------------------------------
(4327, 50, 5) (4327,)
(10, 50, 5) (10,)
(5, 50, 5) (5,)
-------------------------------
[ # , timestep, feature수 ] <-- 데이터의 구조 : 


데이터의 구조 : [n, timestep, feature수]

# 같이 해보기
**[참고링크](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional)**
1. 적절한 인풋 레이어를 구성할 것
2. Conv1D
    * 16개 필터 사용할 것
    * window_size(filter_size)는 5일
    * activation은 swish
3. MaxPool1D
    * 필터 사이즈(window size)는 2일
4. Bidirectional 레이어
    * LSTM, 히든스테이트 노드 32개
5. Bidirectional 레이어
    * LSTM, 히든스테이트 노드 32개
4. 플래튼
5. 회귀를 위한 적절한 아웃풋 레이어



In [88]:
x_train.shape, y_train.shape

((4327, 50, 5), (4327,))

In [89]:
import tensorflow
from tensorflow import keras
from keras.layers import Input, Dense, Flatten, Conv1D, MaxPool1D, Bidirectional, LSTM
from keras.models import Model
from keras.backend import clear_session

from sklearn.metrics import mean_squared_error as MSE

In [90]:
####################
## Your Code Here ##
####################
# 1. 적절한 인풋 레이어를 구성할 것
clear_session()
# 2. Conv1D
#     * 16개 필터 사용할 것
#     * window_size(filter_size)는 5일
#     * activation은 swish
il = Input(shape = (50, 5))
hl = Conv1D(filters = 16, kernel_size = 5, activation = 'swish')(il)
# 3. MaxPool1D
#     * 필터 사이즈(window size)는 2일
hl = MaxPool1D(2)((hl))
# 4. Bidirectional 레이어
#     * LSTM, 히든스테이트 노드 32개
hl = Bidirectional(layer = LSTM(32, return_sequences=True))(hl)
# 5. Bidirectional 레이어
#     * LSTM, 히든스테이트 노드 32개
hl = Bidirectional(layer = LSTM(32, return_sequences=True))(hl)
# 4. 플래튼
hl = Flatten()(hl)
# 5. 회귀를 위한 적절한 아웃풋 레이어
ol = Dense(1, activation = 'relu')(hl)

model = Model(il, ol)

model.compile(loss = 'mae', optimizer = 'adam')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50, 5)]           0         
                                                                 
 conv1d (Conv1D)             (None, 46, 16)            416       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 23, 16)           0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, 23, 64)           12544     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 23, 64)           24832     
 nal)                                                            
                                                             

In [91]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   patience = 7,
                   restore_best_weights = True)

In [92]:
## 학습도 시킬 것
model.fit(x_train, y_train, epochs = 1000, callbacks = [es], validation_data=(x_val, y_val), verbose = 1)


Epoch 1/1000
136/136 [==============================] - 15s 52ms/step - loss: 10.1912 - val_loss: 27.5330
Epoch 2/1000
136/136 [==============================] - 6s 47ms/step - loss: 2.1966 - val_loss: 7.1785
Epoch 3/1000
136/136 [==============================] - 5s 40ms/step - loss: 1.7611 - val_loss: 2.9052
Epoch 4/1000
136/136 [==============================] - 7s 49ms/step - loss: 1.4768 - val_loss: 4.6736
Epoch 5/1000
136/136 [==============================] - 6s 45ms/step - loss: 1.5227 - val_loss: 3.9167
Epoch 6/1000
136/136 [==============================] - 7s 50ms/step - loss: 1.4589 - val_loss: 3.2964
Epoch 7/1000
136/136 [==============================] - 6s 41ms/step - loss: 1.6458 - val_loss: 3.3612
Epoch 8/1000
136/136 [==============================] - 7s 48ms/step - loss: 1.3593 - val_loss: 5.3199
Epoch 9/1000
136/136 [==============================] - 6s 41ms/step - loss: 1.3058 - val_loss: 2.7119
Epoch 10/1000
136/136 [==============================] - 7s 48ms/step 

In [93]:
y_pred = model.predict(x_test)

1/1 [==============================] - 2s 2s/step


In [94]:
# Q1. 테스트 셋에서의 RMSE를 출력하여라.
RMSE = MSE(y_test, y_pred) ** 0.5
print(RMSE)

0.9124057472463845


## Q1. Conv1D + Bidirectional

**[공식문서 적극 참고](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional)**
Bidirectional)**
1. 적절한 인풋 레이어를 구성할 것
2. Conv1D
    * 32개 필터 사용할 것
    * window_size(filter_size)는 10일
    * activation은 swish
3. MaxPool1D
    * 필터 사이즈(window size)는 2일
4. Bidirectional 레이어
    * forward layer : LSTM, 히든스테이트 노드 24개
    * backward layer : GRU, 히든스테이트 노드 16개
4. Bidirectional 레이어
    * forward layer : LSTM, 히든스테이트 노드 24개
    * backward layer : GRU, 히든스테이트 노드 24개
4. 플래튼
6. Fully Connected : 노드 256개, swish
5. 회귀를 위한 적절한 아웃풋 레이어


In [95]:
x_train.shape, y_train.shape

((4327, 50, 5), (4327,))

In [96]:
import tensorflow
from tensorflow import keras
from keras.layers import Input, Dense, Flatten, Conv1D, MaxPool1D, Bidirectional, LSTM, GRU
from keras.models import Model
from keras.backend import clear_session

from sklearn.metrics import mean_squared_error as MSE

In [97]:
def backward_layers(nodes) :
    GRU(nodes, return_sequences = True, go_backwards=True)

In [98]:
####################
## Your Code Here ##
####################
clear_session()
# 1. 적절한 인풋 레이어를 구성할 것
il = Input(shape = (50, 5))
# 2. Conv1D
#     * 32개 필터 사용할 것
#     * window_size(filter_size)는 10일
#     * activation은 swish
hl = Conv1D(filters = 32, kernel_size = 10, activation = 'swish')(il)
# 3. MaxPool1D
#     * 필터 사이즈(window size)는 2일
hl = MaxPool1D(2)(hl)
# 4. Bidirectional 레이어
forward_layers = LSTM(24, return_sequences = True)
#     * forward layer : LSTM, 히든스테이트 노드 24개
#     * backward layer : GRU, 히든스테이트 노드 16개
hl = Bidirectional(layer =forward_layers, backward_layer=backward_layers(16))(hl)
# 4. Bidirectional 레이어
#     * forward layer : LSTM, 히든스테이트 노드 24개
#     * backward layer : GRU, 히든스테이트 노드 24개
hl = Bidirectional(layer =forward_layers, backward_layer=backward_layers(24))(hl)
# 4. 플래튼
hl = Flatten()(hl)
# 6. Fully Connected : 노드 256개, swish
hl = Dense(256, activation = 'swish')(hl)
# 5. 회귀를 위한 적절한 아웃풋 레이어
ol = Dense(1, activation = 'relu')(hl)

model = Model(il, ol)

model.compile(loss = 'mae', optimizer = 'adam')

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50, 5)]           0         
                                                                 
 conv1d (Conv1D)             (None, 41, 32)            1632      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 20, 32)           0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, 20, 48)           10944     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 20, 48)           14016     
 nal)                                                            
                                                             

In [99]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   patience = 7,
                   restore_best_weights = True)

In [100]:
## 학습도 시킬 것
model.fit(x_train, y_train, epochs = 1000, callbacks = [es], validation_data=(x_val, y_val), verbose = 1)



Epoch 1/1000
136/136 [==============================] - 17s 60ms/step - loss: 6.4090 - val_loss: 4.1847
Epoch 2/1000
136/136 [==============================] - 5s 40ms/step - loss: 2.3650 - val_loss: 5.5310
Epoch 3/1000
136/136 [==============================] - 6s 47ms/step - loss: 2.3871 - val_loss: 8.1243
Epoch 4/1000
136/136 [==============================] - 6s 40ms/step - loss: 2.2427 - val_loss: 1.1494
Epoch 5/1000
136/136 [==============================] - 6s 47ms/step - loss: 1.8928 - val_loss: 0.6974
Epoch 6/1000
136/136 [==============================] - 6s 40ms/step - loss: 3.0992 - val_loss: 0.6569
Epoch 7/1000
136/136 [==============================] - 6s 47ms/step - loss: 2.2570 - val_loss: 6.1853
Epoch 8/1000
136/136 [==============================] - 8s 56ms/step - loss: 2.2733 - val_loss: 3.6199
Epoch 9/1000
136/136 [==============================] - 7s 48ms/step - loss: 2.0521 - val_loss: 3.1921
Epoch 10/1000
136/136 [==============================] - 6s 41ms/step - 

In [101]:
y_pred = model.predict(x_test)

1/1 [==============================] - 2s 2s/step


In [102]:
# Q1. 테스트 셋에서의 RMSE를 출력하여라.
RMSE = MSE(y_test, y_pred) ** 0.5
print(RMSE)


0.6505188160753469
